****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
data = json.load((open('data/world_bank_projects.json')))

### 1. Find the 10 countries with the most projects
The strategy here is to use json_normalize to create a DataFrame that captures
the 'countryshortname' attribute.  The .value_counts() DataFrame method is used
with slicing to obtain the top ten countries with the most projects.

In [3]:
df_countries = json_normalize(data, '_id', 'countryshortname')

print("Top 10 countries with the most projects\n")
print(df_countries.countryshortname.value_counts()[0:10])

Top 10 countries with the most projects

China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Bangladesh            12
Morocco               12
Nepal                 12
Mozambique            11
Africa                11
Name: countryshortname, dtype: int64


### 2. Find the top 10 major project themes (using column 'mjtheme_namecode')
Using json_normalize, the project themes are extracted in to a DataFrame and then .value_counts()
is used to extract the top ten project themes.  Notice that there is a blank area which indicates
that some of the project themes are not listed properly in the data.  This will be cleaned up
in the next section.

In [4]:
df_themes = json_normalize(data, 'mjtheme_namecode')

print("Top 10 major project themes\n")
print(df_themes.name.value_counts()[0:10])

# Check to see if there are any blanks in the name column (expect an error here)
try:
    assert sum(df_themes.name == '') == 0
    print("The DataFrame has no blanks in the name column!\n")
except AssertionError:
    print("\nThe DataFrame has blanks in the name column!")

Top 10 major project themes

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

The DataFrame has blanks in the name column!


### 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.
I approached this problem originally using brute force.  I created a hard-coded list of all
the codes in the project DataFrame and a list of all the project themes.  I then zipped these
in to a dictionary, looped through each of the codes, and assigned to any row with a missing name
the correct name that matched with the code in that row.  There is nothing wrong with this approach
since there were only 11 different codes.  If there were many more, then this would cause issues.
Imagine having 100 project themes -- finding all of them and hard-coding these values would defeat
the purpose of using code to complete this task.

The next approach I took was to figure out a way to automatically map the codes to the names by
searching through the DataFrame.  Since I knew the codes were integers, I decided that I could
group by the different project themes and use the mean of the codes to create a new table with
each distinct name and code.  This only works since the codes could be converted to integers and 
with the assumption that every code and name was correct.  If one name was matched with an incorrect
code, then this method would have issues.  I then used the result of the grouping to create a dictionary
mapping of the codes and themes.

A third approach I thought of was to drop any duplicates out of the table that contained the codes
and themes.  This is again based on the assumption that the codes and themes were all matched up correctly.
This method removed the need to rely on the codes being integers and could be extended to other columns that
contained alphanumeric strings.

In [5]:
df_themes_fixed = df_themes.copy()

# The method described below was how the original mapping was created
# to fill in the missing names based on the codes

# Manually get the name string for each of the codes using the 
# following line of code, then create a dictionary mapping the codes
# to the name strings
# print(df_themes_fixed[df_themes_fixed['code'] == '11'])
#codes = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
#names = ['Economic management', "Public sector governance", "Rule of law",
#        "Financial and private sector development", "Trade and integration",
#        "Social protection and risk management", "Social dev/gender/inclusion",
#        "Human development", "Urban development", "Rural development", 
#        "Environment and natural resources management"]
#codes_dict = dict(zip(codes, names))

# The following method is a programmatic way to fill in the names
# based on the existing codes in the table.  This would be the preferred
# method if there were significantly more codes than the 11 that are in
# the current form of the data.  Also, if the codes were not integers,
# another method would need to be used as well.

# Generate a list of all of the codes in the DataFrame
codes_list = list(df_themes_fixed.code.unique())

# Convert the current codes to integers
df_themes_fixed['code'] = df_themes_fixed['code'].astype(int)

codes_dict = {}

# Loop through all of the unique codes
for code in codes_list:
    # For each code, find all of the rows that are not empty
    # Assume that each code is assigned to the correct name
    # Aggregate using the mean to get the original code after grouping by name
    df_code = df_themes_fixed.loc[(df_themes_fixed['code'] == int(code)) & 
                                  (df_themes_fixed['name'] != ''), :].groupby('name').mean()
    
    codes_dict[df_code.code[0]] = df_code.index[0]

# Originally defined a new function to return the name from the dictionary
#def get_name(row):
#    return codes_dict[row['code']]

# Select rows where the name is empty and then apply the lambda function to
# retrieve the name corresponding to the code
#df_themes_fixed.loc[df_themes_fixed['name'] == '', 'name'] = \
#                               df_themes_fixed.apply(get_name, axis = 1)
df_themes_fixed.loc[df_themes_fixed['name'] == '', 'name'] = \
                                df_themes_fixed.apply(lambda r: codes_dict[r['code']], 
                                                      axis = 1)

# Check to see if there are any blanks in the name column (expect no error here)
try:
    assert sum(df_themes_fixed.name == '') == 0
    print("The DataFrame has no blanks in the name column!\n")
except AssertionError:
    print("\nThe DataFrame has blanks in the name column!")

# Print out a clean list of the top 10 major project themes
print("Top 10 major project themes\n")
print(df_themes_fixed.name.value_counts()[0:10])
print("\nTop 10 codes (should match the preceding list)\n")
print(df_themes_fixed.code.value_counts()[0:10])

The DataFrame has no blanks in the name column!

Top 10 major project themes

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64

Top 10 codes (should match the preceding list)

11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: code, dtype: int64


In [6]:
# A third way to extract names associated with each code.
# This method would work on more types of data since it does
# not rely on the code column being an integer
df_test = df_themes.copy()
df_test.head()

df_no_dupes = df_test.loc[df_test['name'] != '', :].drop_duplicates()

codes_list2 = list(df_no_dupes.code)

codes_dict2 = {}
for code in codes_list2:
        codes_dict2[code] = df_no_dupes.loc[df_no_dupes.code == code, 'name'].iloc[0]


# One could use this dictionary in the same manner as above
# I used a new copy of the themes DataFrame (df_test)
df_test.loc[df_test['name'] == '', 'name'] = \
                                df_test.apply(lambda r: codes_dict2[r['code']], axis = 1)

# Check to see if there are any blanks in the name column (expect no error here)
try:
    assert sum(df_test.name == '') == 0
    print("The DataFrame has no blanks in the name column!\n")
except AssertionError:
    print("\nThe DataFrame has blanks in the name column!")

# Print out a clean list of the top 10 major project themes
print("Top 10 major project themes\n")
print(df_test.name.value_counts()[0:10])
print("\nTop 10 codes (should match the preceding list)\n")
print(df_test.code.value_counts()[0:10])

The DataFrame has no blanks in the name column!

Top 10 major project themes

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64

Top 10 codes (should match the preceding list)

11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: code, dtype: int64
